# Shortest path

In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import json

## Récuperer toutes les gares de France

In [2]:
dfStop = pd.read_csv('../data_sncf/stops.txt', sep=",")
dfStop.shape

(9176, 9)

In [3]:
dfStop = dfStop[dfStop['stop_id'].str.contains('StopPoint:OCETrain')]
dfStop.shape

(2447, 9)

In [4]:
allGare = dfStop['stop_name'].unique()
print(allGare)

['Gare de Paris-St-Lazare' 'Gare de Rouen-Rive-Droite' 'Gare de Montville'
 ... 'Gare de Novillars' 'Gare de Deluz' 'Gare de Laissey']


## Extractons nos voisins

In [5]:
dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")

/var/folders/0l/4rhc85qd4kl41w3sd26dml3h0000gn/T/ipykernel_51894/3309677073.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")


In [6]:
def str_to_time(time: str):
    time_split = time.split(':')
    if time_split[0] > '23':
        time_split[0] = str(int(time_split[0]) - 24)
    time = f'{time_split[0]}:{time_split[1]}:{time_split[2]}'
    time_object = datetime.strptime(time, '%H:%M:%S')
    return time_object

In [7]:
def getVoisins(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for i in range(len(trajets)):
        # on recupere la position de la gare dans le trajet
        stopSequence = trajets['stop_sequence'].values[i]
        # on recupere le tripID du trajet
        tripId = trajets['trip_id'].values[i]
        # on recupere les times de notre gare
        departureTimeGare = str_to_time(trajets['departure_time'].values[i])
        arivalTimeGare = str_to_time(trajets['arrival_time'].values[i])

        try:
            # on recupere la gare précédente
            voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence - 1)]
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = str_to_time(voisin1.departure_time.values[0])
            # on recupere le nom de la gare
            voisin1 = dfStop[dfStop['stop_id'] ==
                             voisin1.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue

        try:
            # on recupere la gare suivante
            voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                dfStopTimes['stop_sequence'] == stopSequence + 1)]
            # on recupere le departure time de la gare suivante
            arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.values[0])
            # on recupere le nom de la gare
            voisin2 = dfStop[dfStop['stop_id'] ==
                             voisin2.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue

    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare


In [8]:
graph = {}
for i in tqdm(range(len(allGare))):
    graph[allGare[i]] = getVoisins(allGare[i])
print(graph)
# print(getVoisins('Gare de Neufchâteau'))


100%|██████████| 2447/2447 [38:02<00:00,  1.07it/s]  

{'Gare de Paris-St-Lazare': {'Gare de Rouen-Rive-Droite': 5220, 'Gare de Vernon-Giverny': 3240, 'Gare de Mantes-la-Jolie': 2340, 'Gare de Caen': 7320, 'Gare de Evreux - Normandie': 5280, 'Gare de Lisieux': 6000}, 'Gare de Rouen-Rive-Droite': {'Gare de Paris-St-Lazare': 5220, 'Gare de Montville': 720, 'Gare de Yvetot': 1260, 'Gare de Oissel': 600, 'Gare de Le Havre': 2760, 'Gare de Elbeuf-St-Aubin': 960, 'Gare de Morgny': 900, 'Gare de Longuerue-Vieux-Manoir': 1080, 'Gare de Montérolier-Buchy': 1440, 'Gare de Barentin': 780, 'Gare de Sotteville': 360, 'Gare de Maromme': 420}, 'Gare de Montville': {'Gare de Rouen-Rive-Droite': 720, 'Gare de Clères': 240, 'Gare de Malaunay-le-Houlme': 360}, 'Gare de Clères': {'Gare de Montville': 300, 'Gare de Auffay': 600, 'Gare de St-Victor': 420}, 'Gare de Auffay': {'Gare de Clères': 660, 'Gare de Dieppe': 900, 'Gare de St-Victor': 360, 'Gare de Longueville-sur-Scie': 540}, 'Gare de Dieppe': {'Gare de Auffay': 1020, 'Gare de St-Aubin-sur-Scie': 360}, '

In [9]:
# load json module
import json

# create json object from dictionary
json = json.dumps(graph)

# open file for writing, "w" 
f = open("graph.json","w")

# write json object to file
f.write(json)

# close file
f.close()